In [1]:
import nltk
import numpy as np
import random
import string

### Definir CORPUS

In [2]:
f = open("corpus.txt","r",errors='ignore')
raw = f.read()

In [3]:
raw

'En cuarentena no se pude salir de casa.\n\ncomo medidas de prevencion, recuerde siempre laverse las manos y utilizar tababocas.\n\nlos tapabocas los puede comprar en la farmacia.\n\nPuede comprar alimentos a traves de domicilio.\n\nNo se tiene prevista fecha de vacunacion en Colombia.\n\nLos medicamentos los puede comprar a domicilio.\n\n\n'

### Preporcesamiento del texto con NLTK CORPUS

In [4]:
raw = raw.lower() #convertir en minuscular
sent_tokens = nltk.sent_tokenize(raw) # convertir corpus a una lista de sentencias (tekenizar en sentencias)
word_tokens = nltk.word_tokenize(raw) # convertir corpus a una lista de palabras (tokenizar en palabras)

lemmer = nltk.stem.WordNetLemmatizer()#

In [10]:
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation) #remover signos de puntuacion (utilizamos la )

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens] # Lematizar palabras tokenizadas

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [27]:
# remove_punct_dict = dict((ord(punct),word) for punct in string.punctuation for word in word_tokens)

### Evaluar Similitud Mensaje Usuario - Corpus

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords  

In [55]:
#Determinar la similutud del texto inserdado y el corpus

def response(user_response):# funcion respuesta (entrada: mensje de usuario)
    chatbot_response = '' #definir la respuesta del chatbot
    sent_tokens.append(user_response)# al listado de sentencias del corpus añadir al final de la lista el mensaje del usuario
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize,stop_words=stopwords.words('spanish'))# Lematizar y eliminar palbras
    tfidf = TfidfVec.fit_transform(sent_tokens)

    print("a",tfidf[-1])
    print("------------------")
    print(tfidf)
    # 3 Evaluar similitud coseno entre mensaje usuario (tfid[-1]) y el corpus(tfidf)
    vals = cosine_similarity(tfidf[-1],tfidf)
   

    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    
    flat.sort()
  
    req_tfidf = flat[-2]
    print("------------------")
    print(req_tfidf)
    print(idx)
    if(req_tfidf == 0):
        chatbot_response = chatbot_response + "Lo siento, no te he entendido"
        return chatbot_response
    
    else:
        chatbot_response = chatbot_response + sent_tokens[idx]
        return chatbot_response

### DEFINICION DE CONINCIDENCIA MANUAL

In [56]:
SALUDOS_INPUTS = ("hola","buenas","saludos","que tal","hey","buenos dias")
SALUDOS_OUTPUTS = ["hola","hola, que tal?","Hola, Como te puedo ayudar?","hola, encantado de hablar contigo"]

def saludos(sentence):
    for word in sentence.split():
        # print(word.lower() in SALUDOS_INPUTS)
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

### GENERACION DE RESPUESTA

In [57]:
flag = True

print("AGENTE: MI NOMBRE ES AGENTE. CONTESTARE A SUS PREGUNTAS ACERCA DEL COVID Y LAS MEDIDAS DE SIGUIRDAD.")
while(flag == True):
    user_response = input()
    user_response = user_response.lower() #convertimos a minuscula

    if(user_response not in ["salir","chao","adios"]):
        if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
            flag = True
            print("AGENTE: No hay de que")
        else:
            if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
                print("AGENTE: "+saludos(user_response))
            else:#si la palabra insertada no es un saludo --> Corpus
                print("AGENTE: ",end ="*")
                print(response(user_response),end="*\n")
                sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
    else:
        flag = False
        print("AGENTE: Nos vemos pronto, !Gracias por su visita¡")                
    

AGENTE: MI NOMBRE ES AGENTE. CONTESTARE A SUS PREGUNTAS ACERCA DEL COVID Y LAS MEDIDAS DE SIGUIRDAD.
AGENTE: *a   (0, 3)	1.0
------------------
  (0, 1)	0.5
  (0, 17)	0.5
  (0, 14)	0.5
  (0, 4)	0.5
  (1, 19)	0.35355339059327373
  (1, 22)	0.35355339059327373
  (1, 9)	0.35355339059327373
  (1, 8)	0.35355339059327373
  (1, 18)	0.35355339059327373
  (1, 16)	0.35355339059327373
  (1, 12)	0.35355339059327373
  (1, 11)	0.35355339059327373
  (2, 6)	0.6169266931593301
  (2, 3)	0.3911811902785459
  (2, 15)	0.44615681259330786
  (2, 20)	0.5170327167755766
  (3, 5)	0.4400320599888547
  (3, 21)	0.5250490246458531
  (3, 0)	0.5250490246458531
  (3, 3)	0.33292335166069675
  (3, 15)	0.37971156360828673
  (4, 2)	0.5
  (4, 23)	0.5
  (4, 7)	0.5
  (4, 13)	0.5
  (5, 10)	0.6169266931593301
  (5, 5)	0.5170327167755766
  (5, 3)	0.3911811902785459
  (5, 15)	0.44615681259330786
  (6, 20)	1.0
  (7, 3)	1.0
------------------
0.391181190278546
5
los medicamentos los puede comprar a domicilio.*
AGENTE: Nos vemos pro

In [49]:
sent_tokens

['en cuarentena no se pude salir de casa.',
 'como medidas de prevencion, recuerde siempre laverse las manos y utilizar tababocas.',
 'los tapabocas los puede comprar en la farmacia.',
 'puede comprar alimentos a traves de domicilio.',
 'no se tiene prevista fecha de vacunacion en colombia.',
 'los medicamentos los puede comprar a domicilio.',
 'tapabocas']